In [ ]:
from huggingface_hub import login
token = ""
login(token)

from datasets import load_dataset
ds = load_dataset("diarizers-community/callhome", "eng", )
print(ds)

sample = ds['data'][0]['audio']['array']
noise = ds['data'][1]['audio']['array']
rate = ds['data'][0]['audio']['sampling_rate']

# save audio
import soundfile as sf
sf.write("data/sample.wav", sample, rate)

# play audio
import IPython.display as ipd
ipd.Audio(sample, rate=rate)

In [27]:
import numpy as np
import librosa
import soundfile as sf
import requests
import os

def add_noise_snr(clean_audio, noise_audio, snr_db):
    """Add noise to clean audio at a specified SNR (dB)."""
    
    # Ensure noise is at least as long as clean speech
    if len(noise_audio) < len(clean_audio):
        noise_audio = np.tile(noise_audio, int(np.ceil(len(clean_audio) / len(noise_audio))))
    noise_audio = noise_audio[:len(clean_audio)]
    
    # Calculate signal power and noise power
    signal_power = np.mean(clean_audio ** 2)
    noise_power = np.mean(noise_audio ** 2)
    
    # Scale noise to achieve the desired SNR
    desired_noise_power = signal_power / (10 ** (snr_db / 10))
    scaling_factor = np.sqrt(desired_noise_power / noise_power)
    scaled_noise = noise_audio * scaling_factor
    
    # Add scaled noise to clean signal
    noisy_audio = clean_audio + scaled_noise
    
    # Normalize to prevent clipping
    max_val = max(np.abs(noisy_audio).max(), 1.0)
    noisy_audio = noisy_audio / max_val
    
    return noisy_audio

# Load clean speech
clean_audio, sr = sample, rate

# Download and load noise
noise_audio = noise

for snr in range(20):
    if snr < 5: 
        snr = 2**snr
    else: 
        snr = 16 + snr
    # Add noise with specified SNR
    noisy_sample = add_noise_snr(clean_audio, noise_audio, snr)
    # save audio
    sf.write(f"data/oa/noisy_sample_snr{snr}.wav", noisy_sample, rate)

# # play audio
# import IPython.display as ipd
# ipd.Audio(noisy_sample, rate=rate)

# # Print speaker timestamps
# for turn, _, speaker in diarization.itertracks(yield_label=True):
#     print(f"Speaker {speaker}: {turn.start:.2f}s - {turn.end:.2f}s")

# with open("output.rttm", "w") as rttm_file:
#     diarization.write_rttm(rttm_file)

In [28]:
import numpy as np
import librosa
import soundfile as sf
import requests
import os

def add_noise_snr(clean_audio, noise_audio, snr_db):
    """Add noise to clean audio at a specified SNR (dB)."""
    
    # Ensure noise is at least as long as clean speech
    if len(noise_audio) < len(clean_audio):
        noise_audio = np.tile(noise_audio, int(np.ceil(len(clean_audio) / len(noise_audio))))
    noise_audio = noise_audio[:len(clean_audio)]
    
    # Calculate signal power and noise power
    signal_power = np.mean(clean_audio ** 2)
    noise_power = np.mean(noise_audio ** 2)
    
    # Scale noise to achieve the desired SNR
    desired_noise_power = signal_power / (10 ** (snr_db / 10))
    scaling_factor = np.sqrt(desired_noise_power / noise_power)
    scaled_noise = noise_audio * scaling_factor
    
    # Add scaled noise to clean signal
    noisy_audio = clean_audio + scaled_noise
    
    # Normalize to prevent clipping
    max_val = max(np.abs(noisy_audio).max(), 1.0)
    noisy_audio = noisy_audio / max_val
    
    return noisy_audio

# Load clean speech
clean_audio, sr = sample, rate

# Load audio file
import librosa
import numpy as np
noise_audio, sr = librosa.load("data/noise.mp3", sr=16000)

for snr in range(20):
    if snr < 5: 
        snr = 2**snr
    else: 
        snr = 16 + snr
    # Add noise with specified SNR
    noisy_sample = add_noise_snr(clean_audio, noise_audio, snr)
    # save audio
    sf.write(f"data/na/noisy_sample_snr{snr}.wav", noisy_sample, rate)